In [224]:
%run ml_header.py

In [2]:
es_v4 = pd.read_csv('exhaustive_training_v4.csv')

In [3]:
# direct mapping
dm_v4 = pd.read_csv('direct_mapping_v4.csv')
dm_v5 = pd.read_csv('direct_mapping_v5.csv')

In [4]:
# direct mapping with tolerance
dm_v4_01 = pd.read_csv('direct_mapping_v4_tolerance_0.1.csv')
dm_v5_01 = pd.read_csv('direct_mapping_v5_tolerance_0.1.csv')
dm_v4_005 = pd.read_csv('direct_mapping_v4_tolerance_0.05.csv')
dm_v5_005 = pd.read_csv('direct_mapping_v5_tolerance_0.05.csv')

# For FPGA v4

In [225]:
benchmarks = list(dm_v4['benchmark'])
metrics_moo = list(dm_v4)[1:]
metrics_moo

['adrs_ave',
 'adrs_max',
 'adrs_ave_rms',
 'adrs_max_rms',
 'hypervolume',
 'dominance',
 'cardinality']

## Without tolerance

In [226]:
similarity = dict()
for benchmark in benchmarks:
    similarity[benchmark] = dict(zip(benchmarks, [0] * len(benchmarks)))
    dm_metrics = dm_v4[dm_v4['benchmark'].str.contains(benchmark)][metrics_moo[:-1]].values
    es_v4_benchmark = es_v4[es_v4['test'].str.contains(benchmark)]
    adrs_bool = es_v4_benchmark[metrics_moo[:4]].values <= dm_metrics[:, :4]
    others_bool = es_v4_benchmark[metrics_moo[4:-1]].values >= dm_metrics[:, 4:]
    adrs_bool = np.all(adrs_bool, axis=1)
    others_bool = np.all(others_bool, axis=1)
    final_bool = np.all(np.c_[adrs_bool, others_bool], axis=1)
    good_combinations = es_v4_benchmark[final_bool]
    for combination in good_combinations['train']:
        combo_list = combination.split(';')
        for k in combo_list:
            similarity[benchmark][k] += 1

In [227]:
simi_mat = pd.DataFrame(0, index=benchmarks, columns=benchmarks)
simi_mat

,adpcm_encoder,aes,ann,average,decimation,fft_fixed,fir,idct,interpolation,kasumi,qsort,snow3g,sobel
adpcm_encoder,0,0,0,0,0,0,0,0,0,0,0,0,0
aes,0,0,0,0,0,0,0,0,0,0,0,0,0
ann,0,0,0,0,0,0,0,0,0,0,0,0,0
average,0,0,0,0,0,0,0,0,0,0,0,0,0
decimation,0,0,0,0,0,0,0,0,0,0,0,0,0
fft_fixed,0,0,0,0,0,0,0,0,0,0,0,0,0
fir,0,0,0,0,0,0,0,0,0,0,0,0,0
idct,0,0,0,0,0,0,0,0,0,0,0,0,0
interpolation,0,0,0,0,0,0,0,0,0,0,0,0,0
kasumi,0,0,0,0,0,0,0,0,0,0,0,0,0


In [229]:
for target, sources in similarity.items():
    for benchmark, count in sources.items():
        simi_mat.loc[benchmark, target] = count

simi_mat

,adpcm_encoder,aes,ann,average,decimation,fft_fixed,fir,idct,interpolation,kasumi,qsort,snow3g,sobel
adpcm_encoder,0,266,42,1214,1044,20,1323,1364,351,275,3,167,452
aes,39,0,44,1187,1075,4,1327,1332,352,277,8,151,454
ann,9,0,0,617,656,0,1506,1385,532,0,0,68,141
average,27,258,48,0,1063,21,1290,1315,363,255,5,161,505
decimation,35,267,54,1192,0,21,1305,1339,383,270,9,153,481
fft_fixed,63,435,96,1219,1526,0,1447,1597,539,338,16,324,677
fir,33,264,56,1194,1048,17,0,1306,354,254,6,154,481
idct,0,1,0,1095,1183,27,1089,0,566,264,0,198,712
interpolation,6,255,18,1312,1092,24,1388,1001,0,10,0,252,448
kasumi,44,207,65,1215,1037,0,1240,1261,271,0,16,116,466


In [232]:
simi_mat_t = simi_mat.transpose()
simi_space = simi_mat + simi_mat_t
simi_space

,adpcm_encoder,aes,ann,average,decimation,fft_fixed,fir,idct,interpolation,kasumi,qsort,snow3g,sobel
adpcm_encoder,0,305,51,1241,1079,83,1356,1364,357,319,26,173,472
aes,305,0,44,1445,1342,439,1591,1333,607,484,272,361,715
ann,51,44,0,665,710,96,1562,1385,550,65,37,68,176
average,1241,1445,665,0,2255,1240,2484,2410,1675,1470,1191,1495,1649
decimation,1079,1342,710,2255,0,1547,2353,2522,1475,1307,1157,1120,1544
fft_fixed,83,439,96,1240,1547,0,1464,1624,563,338,20,330,697
fir,1356,1591,1562,2484,2353,1464,0,2395,1742,1494,1360,1401,1787
idct,1364,1333,1385,2410,2522,1624,2395,0,1567,1525,1337,1496,2017
interpolation,357,607,550,1675,1475,563,1742,1567,0,281,234,526,837
kasumi,319,484,65,1470,1307,338,1494,1525,281,0,275,394,696


Clustering

In [261]:
import scipy.cluster.hierarchy as hcluster
thresh = [500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
for i in thresh:
    clusters = hcluster.fclusterdata(simi_space.as_matrix(), i, criterion='distance')
    print(i, clusters)

500 [ 1  3  7 10  9  4 11 12  6  5  1  2  8]
600 [ 1  2  6  9  8  3 10 11  5  4  1  1  7]
700 [1 1 3 6 5 1 7 8 2 1 1 1 4]
800 [1 1 2 5 4 1 6 7 1 1 1 1 3]
900 [1 1 1 4 3 1 5 6 1 1 1 1 2]
1000 [1 1 1 4 3 1 5 6 1 1 1 1 2]
1500 [1 1 1 3 2 1 4 5 1 1 1 1 1]
2000 [1 1 1 3 2 1 4 5 1 1 1 1 1]
2500 [1 1 1 3 2 1 4 5 1 1 1 1 1]
3000 [1 1 1 2 1 1 3 4 1 1 1 1 1]
3500 [1 1 1 1 1 1 2 2 1 1 1 1 1]
4000 [1 1 1 1 1 1 1 1 1 1 1 1 1]
4500 [1 1 1 1 1 1 1 1 1 1 1 1 1]
5000 [1 1 1 1 1 1 1 1 1 1 1 1 1]


In [246]:
kmeans_model = KMeans(n_clusters=3, random_state=42).fit(simi_space.as_matrix())
labels = kmeans_model.labels_
labels

array([1, 1, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 1])

In [244]:
for n in range(2, len(benchmarks)):
    kmeans_model = KMeans(n_clusters=n, random_state=42).fit(simi_space.as_matrix())
    labels = kmeans_model.labels_
    print(n, metrics.calinski_harabaz_score(simi_space.as_matrix(), labels))

2 14.9873097652
3 12.1109521027
4 12.9169269731
5 19.0279425235
6 21.5079686292
7 24.5106333233
8 30.6672138871
9 33.3632143702
10 36.0537329965
11 47.0562802298
12 268.578628265


For a benchmark as testing, find the benchmarks as training set that leads to better results than direct mapping.

In [152]:
benchmark = benchmarks[11]

dm_metrics = dm_v4[dm_v4['benchmark'].str.contains(benchmark)][metrics_moo[:-1]].values

es_v4_benchmark = es_v4[es_v4['test'].str.contains(benchmark)]
adrs_bool = es_v4_benchmark[metrics_moo[:4]].values <= dm_metrics[:, :4]
others_bool = es_v4_benchmark[metrics_moo[4:-1]].values >= dm_metrics[:, 4:]

adrs_bool = np.all(adrs_bool, axis=1)

others_bool = np.all(others_bool, axis=1)

final_bool = np.all(np.c_[adrs_bool, others_bool], axis=1)

In [153]:
dm_v4[dm_v4['benchmark'] == benchmark]

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
11,snow3g,0.1039,0.2078,0.0396,0.0793,0.2072,0.5,10.0


In [154]:
es_v4_benchmark[final_bool]

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
440,ann;fft_fixed,snow3g,0.1039,0.2078,0.0183,0.0366,0.2860,0.5,7.0
594,average;qsort,snow3g,0.1039,0.2078,0.0396,0.0793,0.5155,0.5,5.0
737,fft_fixed;interpolation,snow3g,0.1039,0.2078,0.0183,0.0366,0.2835,0.5,8.0
782,fft_fixed;sobel,snow3g,0.1039,0.2078,0.0183,0.0366,0.5135,0.5,7.0
1152,adpcm_encoder;ann;fft_fixed,snow3g,0.1039,0.2078,0.0183,0.0366,0.2860,0.5,7.0
1422,adpcm_encoder;fft_fixed;interpolation,snow3g,0.1039,0.2078,0.0183,0.0366,0.2835,0.5,8.0
1463,adpcm_encoder;fft_fixed;sobel,snow3g,0.1039,0.2078,0.0183,0.0366,0.2835,0.5,8.0
1702,aes;ann;fft_fixed,snow3g,0.1039,0.2078,0.0183,0.0366,0.2860,0.5,7.0
2242,ann;average;fft_fixed,snow3g,0.1039,0.2078,0.0183,0.0366,0.2860,0.5,7.0
2322,ann;decimation;fft_fixed,snow3g,0.1039,0.2078,0.0183,0.0366,0.2860,0.5,7.0


With tolerance

In [159]:
benchmark = benchmarks[12]

dm_metrics = dm_v4_01[dm_v4_01['benchmark'].str.contains(benchmark)][metrics_moo[:-1]].values

es_v4_benchmark = es_v4[es_v4['test'].str.contains(benchmark)]
adrs_bool = es_v4_benchmark[metrics_moo[:4]].values <= dm_metrics[:, :4]
others_bool = es_v4_benchmark[metrics_moo[4:-1]].values >= dm_metrics[:, 4:]

adrs_bool = np.all(adrs_bool, axis=1)

others_bool = np.all(others_bool, axis=1)

final_bool = np.all(np.c_[adrs_bool, others_bool], axis=1)

In [156]:
dm_v4_01[dm_v4_01['benchmark'] == benchmark]

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
12,sobel,0.0,0.0,0.0,0.0,0.1945,1.0,10.0


In [157]:
es_v4_benchmark[final_bool]

,train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
11,adpcm_encoder,sobel,0.0,0.0,0.0,0.0,0.2056,1.0,11.0
23,aes,sobel,0.0,0.0,0.0,0.0,1.0000,1.0,1.0
35,ann,sobel,0.0,0.0,0.0,0.0,0.6479,1.0,5.0
47,average,sobel,0.0,0.0,0.0,0.0,0.6192,1.0,2.0
59,decimation,sobel,0.0,0.0,0.0,0.0,0.3073,1.0,9.0
71,fft_fixed,sobel,0.0,0.0,0.0,0.0,1.0000,1.0,1.0
83,fir,sobel,0.0,0.0,0.0,0.0,0.3376,1.0,3.0
95,idct,sobel,0.0,0.0,0.0,0.0,0.3024,1.0,8.0
107,interpolation,sobel,0.0,0.0,0.0,0.0,1.0000,1.0,1.0
119,kasumi,sobel,0.0,0.0,0.0,0.0,0.2862,1.0,11.0


In [262]:
dm_v4

,benchmark,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality
0,adpcm_encoder,0.0251,0.0978,0.0261,0.0875,0.2722,0.6667,6.0
1,aes,0.0043,0.0216,0.0319,0.1593,0.9669,0.8000,6.0
2,ann,0.0142,0.0600,0.0891,0.5433,0.9356,0.5556,8.0
3,average,0.0000,0.0000,0.0000,0.0000,0.4526,1.0000,8.0
4,decimation,0.0541,0.1482,0.0404,0.1479,0.8890,0.2857,4.0
5,fft_fixed,0.0583,0.1154,0.0383,0.0856,0.7895,0.0000,8.0
6,fir,0.0000,0.0000,0.0000,0.0000,0.3377,1.0000,5.0
7,idct,0.0257,0.0522,0.0281,0.0594,0.7248,0.5000,9.0
8,interpolation,0.2575,0.3985,0.1646,0.2454,0.5816,0.1429,15.0
9,kasumi,0.1375,0.2559,0.1399,0.2621,0.7833,0.2857,4.0
